# 연습문제: 한 번에 하나의 토큰 생성하기

이 연습에서는 LLM이 이전 토큰을 사용하여 다음 토큰을 예측하면서 한 번에 하나의 토큰씩 텍스트를 생성하는 방식을 이해하게 될 것입니다.

## 1단계. 토크나이저와 모델 로드하기

먼저 HuggingFace의 transformers 라이브러리에서 토크나이저와 모델을 로드합니다. 토크나이저는 모델이 이해할 수 있는 숫자 목록으로 문자열을 분할하는 함수입니다.

이 연습에서는 모든 코드가 여러분을 위해 작성될 것입니다. 여러분은 따라하기만 하면 됩니다!


### 설치해야 할 목록
- python3.11 -m venv .venv
- source .venv/bin/activate
- pip install --upgrade pip
- pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
- pip install transformers
- pip install pandas
- pip install IPython
- pip install jupyterlab ipywidgets
- jupyter lab build

### 설치후 실행
- jupyter lab


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# To load a pretrained model and a tokenizer using HuggingFace, we only need two lines of code!
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

# We create a partial sentence and tokenize it.
text = "Udacity is the best place to learn about generative"
inputs = tokenizer(text, return_tensors="pt")

# Show the tokens as numbers, i.e. "input_ids"
inputs["input_ids"]

tensor([[  52,   67, 4355,  318,  262, 1266, 1295,  284, 2193,  546, 1152,  876]])

## 2단계. 토큰화 검토하기

이 토큰들이 무엇을 의미하는지 살펴봅시다!


In [5]:
# Show how the sentence is tokenized
import pandas as pd


def show_tokenization(inputs):
    return pd.DataFrame(
        [(id, tokenizer.decode(id)) for id in inputs["input_ids"][0]],
        columns=["id", "token"],
    )


show_tokenization(inputs)

,id,token
0,tensor(52),U
1,tensor(67),d
2,tensor(4355),acity
3,tensor(318),is
4,tensor(262),the
5,tensor(1266),best
6,tensor(1295),place
7,tensor(284),to
8,tensor(2193),learn
9,tensor(546),about


### 서브워드 토큰화

흥미로운 점은 이 경우 토큰이 단순히 글자도 아니고 단어도 아니라는 것입니다. 때로는 짧은 단어가 단일 토큰으로 표현되지만, 때로는 단일 토큰이 단어의 일부 또는 단일 글자를 나타냅니다. 이를 서브워드 토큰화라고 합니다.

## 2단계. 다음 토큰의 확률 계산하기

이제 PyTorch를 사용하여 이전 토큰이 주어졌을 때 다음 토큰의 확률을 계산해 보겠습니다.


In [6]:
# Calculate the probabilities for the next token for all possible choices. We show the
# top 5 choices and the corresponding words or subwords for these tokens.

import torch

with torch.no_grad():
    logits = model(**inputs).logits[:, -1, :]
    probabilities = torch.nn.functional.softmax(logits[0], dim=-1)


def show_next_token_choices(probabilities, top_n=5):
    return pd.DataFrame(
        [
            (id, tokenizer.decode(id), p.item())
            for id, p in enumerate(probabilities)
            if p.item()
        ],
        columns=["id", "token", "p"],
    ).sort_values("p", ascending=False)[:top_n]


show_next_token_choices(probabilities)

,id,token,p
8300,8300,programming,0.157596
4673,4673,learning,0.148418
4981,4981,models,0.048506
17219,17219,biology,0.046482
16113,16113,algorithms,0.027796


흥미롭네요! 모델은 가장 가능성이 높은 다음 단어가 "programming"이고, 그 다음으로 "learning"이 뒤따른다고 생각합니다.


In [7]:
# Obtain the token id for the most probable next token
next_token_id = torch.argmax(probabilities).item()

print(f"Next token id: {next_token_id}")
print(f"Next token: {tokenizer.decode(next_token_id)}")

Next token id: 8300
Next token:  programming


In [8]:
# We append the most likely token to the text.
text = text + tokenizer.decode(8300)
text

'Udacity is the best place to learn about generative programming'

## 3단계. 토큰을 더 생성하기

다음 셀은 `text`를 가져와서 다음에 나올 가능성이 가장 높은 토큰을 표시하고, 가장 가능성이 높은 토큰을 text에 추가합니다. 셀을 반복해서 실행하여 작동 방식을 확인하세요!


In [9]:
# Press ctrl + enter to run this cell again and again to see how the text is generated.

from IPython.display import Markdown, display

# Show the text
print(text)

# Convert to tokens
inputs = tokenizer(text, return_tensors="pt")

# Calculate the probabilities for the next token and show the top 5 choices
with torch.no_grad():
    logits = model(**inputs).logits[:, -1, :]
    probabilities = torch.nn.functional.softmax(logits[0], dim=-1)

display(Markdown("**Next token probabilities:**"))
display(show_next_token_choices(probabilities))

# Choose the most likely token id and add it to the text
next_token_id = torch.argmax(probabilities).item()
text = text + tokenizer.decode(next_token_id)

Udacity is the best place to learn about generative programming


**Next token probabilities:**

,id,token,p
13,13,.,0.352234
11,11,",",0.135988
290,290,and,0.109375
287,287,in,0.069531
8950,8950,languages,0.058290


## Step 4. Use the `generate` method

In [11]:
from IPython.display import Markdown, display

# Start with some text and tokenize it
text = "Once upon a time, generative models"
inputs = tokenizer(text, return_tensors="pt")

# Use the `generate` method to generate lots of text
output = model.generate(**inputs, max_length=100, pad_token_id=tokenizer.eos_token_id)

# Show the generated text
display(Markdown(tokenizer.decode(output[0])))

Once upon a time, generative models of the human brain were used to study the neural correlates of cognitive function. In the present study, we used a novel model of the human brain to investigate the neural correlates of cognitive function. We used a novel model of the human brain to investigate the neural correlates of cognitive function. We used a novel model of the human brain to investigate the neural correlates of cognitive function. We used a novel model of the human brain to investigate the neural correlates of cognitive function.

옛날 옛적에 인간 두뇌의 생성 모델은 인지 기능의 신경 상관 관계를 연구하는 데 사용되었습니다. 본 연구에서는 인간 두뇌의 새로운 모델을 사용하여 인지 기능의 신경 상관 관계를 조사했습니다. 우리는 인간 두뇌의 새로운 모델을 사용하여 인지 기능의 신경 상관 관계를 조사했습니다.


### 흥미롭네요...

GPT-2는 여러분이 사용 경험이 있을 수 있는 GPT-4와 같은 최신 모델만큼 정교하지 않다는 것을 알 수 있을 것입니다. 종종 스스로 반복하고 항상 말이 되는 것은 아닙니다. 하지만 영어처럼 보이는 텍스트를 생성할 수 있다는 것은 여전히 매우 인상적입니다.

## 연습문제 완료를 축하합니다! 🎉

스스로에게 박수를 보내세요. 그리고 휴식을 취하세요.
